In [ ]:
import numpy as np
import pandas as pd

# pd.set_option('display.height', 1000)
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
pd.set_option("max_colwidth", 200)
from IPython.core.display import HTML, display

display(HTML("<style>.container { width:100% !important; }</style>"))
import os


def df_stats(df):
    from tabulate import tabulate

    print("\n***** Shape: ", df.shape, " *****\n")

    columns_list = df.columns.values.tolist()
    isnull_list = df.isnull().sum().values.tolist()
    isunique_list = df.nunique().values.tolist()
    dtypes_list = df.dtypes.tolist()

    list_stat_val = list(zip(columns_list, isnull_list, isunique_list, dtypes_list))
    df_stat_val = pd.DataFrame(
        list_stat_val, columns=["Name", "Null", "Unique", "Dtypes"]
    )
    print(tabulate(df_stat_val, headers="keys", tablefmt="psql"))
    return df.head()

In [ ]:
import os


def find_dcm_files(directory):
    dcm_files = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".dcm"):
                full_path = os.path.join(root, file)
                dcm_files.append(full_path)

    return dcm_files


# Use the function
directory = "/media/data1/ravram/CathEF_UOHI/"
dcm_files = find_dcm_files(directory)


# Create a DataFrame
df_cathEF_UOHI = pd.DataFrame(dcm_files, columns=["path"])
# df_cathEF_UOHI.to_csv('data/CathEF_Prospective/df_files_UOHI.csv')

# Use the function
directory = "/media/data1/ravram/CathEF_Prospective/"
dcm_files = find_dcm_files(directory)


# Create a DataFrame
df_cathEF_MHI = pd.DataFrame(dcm_files, columns=["path"])
# df_cathEF_MHI.to_csv('data/CathEF_Prospective/df_files_MHI.csv')

## Generate videos

In [ ]:
import numpy as np
import pandas as pd

# pd.set_option('display.height', 1000)
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
pd.set_option("max_colwidth", 200)
from IPython.core.display import HTML, display

display(HTML("<style>.container { width:100% !important; }</style>"))
from downloadAvi import extract_avi_metadata as avi_meta

In [ ]:
import os

directory = "../Oread/CathEF_Prospective/"
avi_files = [file for file in os.listdir(directory) if file.endswith(".avi")]
num_avi_files = len(avi_files)

print(f"Number of .avi files: {num_avi_files}")

In [ ]:
df = pd.read_csv("data/df_files_MHI_UOHI_concat.csv")
# Rename df['dicom_path'] to ['path']
df.rename(columns={"dicom_path": "path"}, inplace=True)
display(df.head(n=5))
# display(df.head(n=5))

df.to_csv("data/df_files_MHI_UOHI_concat.csv")
# row = df[df['FileName'].str.contains('2.16.124.113611.1.118.1.1.6506117_1.3.46.670589.29.1877192777354251333053229336874672.dcm.avi')]
# row.to_csv('data/df_files_MHI_UOHI_concat_test.csv')

In [ ]:
avi_meta.extract_avi_and_metadata(
    "data/df_files_MHI_UOHI_concat.csv",
    data_type="ANGIO",
    destinationFolder="CathEF_Prospective/",
    dataFolder="data/",
)

In [ ]:
df_cathEF_UOHI = pd.read_csv("data/df_files_UOHI.csv_metadata_extracted.csv")
df_cathEF_MHI = pd.read_csv("data/df_files_MHI.csv_metadata_extracted.csv")

df_extracted = pd.read_csv(
    "../pydicom-batch/data/audit-inference-20210803-20230803_extract.csv"
)
df_extracted = df_extracted.groupby(["StudyInstanceUID"]).first().reset_index()
df_extracted_merged = pd.merge(
    df_extracted.groupby(["StudyInstanceUID"]).first().reset_index(),
    df_cathEF_MHI,
    how="right",
    on="StudyInstanceUID",
)
# Rename df_extracted_merged['mrn'] to ['PatientID_anon']
df_extracted_merged.rename(columns={"mrn": "PatientID_anon"}, inplace=True)
# Rename df_extracted_merged['PatientID'] to ['mrn']
df_extracted_merged.rename(columns={"PatientID": "mrn"}, inplace=True)

df_extracted_concat = pd.concat([df_extracted_merged, df_cathEF_UOHI])

display(df_extracted_concat.describe())
df_extracted_concat.to_csv("data/df_files_MHI_UOHI_concat.csv")
display(df_extracted_concat.head(n=5))

In [ ]:
# df_extracted_merged.to_csv('data/CathEF_inference-20231016_FINAL.csv_metadata_extracted.csv')

In [ ]:
# save df_extracted_merged with mu separator
# df_extracted_merged.to_csv('data/CathEF_inference-20231016_FINAL.csv_metadata_extracted_mu.csv', sep='µ')

In [ ]:
import os
import shutil

import pandas as pd
from tqdm import tqdm


def copy_files_with_progress(
    df_extracted_merged, dest_folder="../Oread/CathEF_Prospective/"
):
    # Count and print the number of files in the destination folder before copy
    pre_copy_count = len(os.listdir(dest_folder))
    print(f"Number of files in destination folder before copy: {pre_copy_count}")

    # Iterate through file names in the DataFrame and copy them
    for src_path in tqdm(df_extracted_merged["FileName"]):
        file_name = os.path.basename(src_path)
        dest_path = os.path.join(dest_folder, file_name)

        if os.path.exists(src_path):
            shutil.copy(src_path, dest_path)

    # Count and print the number of files in the destination folder after copy
    post_copy_count = len(os.listdir(dest_folder))
    print(f"Number of files in destination folder after copy: {post_copy_count}")

In [ ]:
df_extracted_merged = pd.read_csv(
    "data/df_files_MHI_UOHI_concat.csv_metadata_extracted.csv"
)
# Drop NaN for df_extracted_merged['dicom_path']
df_extracted_merged.dropna(subset=["dicom_path"], inplace=True)
display(df_extracted_merged.describe())

In [ ]:
copy_files_with_progress(
    df_extracted_merged, dest_folder="../Oread/CathEF_Prospective/"
)

In [ ]:
df_extracted_merged = pd.read_csv(
    "data/df_files_MHI_UOHI_concat.csv_metadata_extracted.csv"
)
# Drop NaN for df_extracted_merged['dicom_path']
df_extracted_merged.dropna(subset=["dicom_path"], inplace=True)

# REname df_extracted_merged['mrn'] to ['PatientID']
df_extracted_merged.rename(columns={"mrn": "PatientID"}, inplace=True)
display(df_extracted_merged.head())

In [ ]:
df_pre_extract = pd.read_csv("data/df_files_MHI_UOHI_concat.csv")
display(df_pre_extract.head(n=5))
df_pre_extract = df_pre_extract[["mrn", "PatientID_anon"]]
df_pre_extract_merged = df_pre_extract.groupby(["PatientID_anon"]).first().reset_index()
df_extracted_merged_m = pd.merge(
    df_extracted_merged,
    df_pre_extract_merged,
    how="left",
    left_on="PatientID",
    right_on="PatientID_anon",
)
## Np.where df_extracted_merged_m['mrn'] is present keep df_extracted_merged_m['PatientID'] is equal to mrn else keep df_extracted_merged_m['PatientID']
df_extracted_merged_m["PatientID"] = np.where(
    df_extracted_merged_m["mrn"].notnull(),
    df_extracted_merged_m["mrn"],
    df_extracted_merged_m["PatientID"],
)
display(df_extracted_merged_m.PatientID.nunique())
df_extracted_merged_m.to_csv("data/df_files_MHI_UOHI_concat_mrn_fixed.csv")

In [ ]:
df_uohi_extracted = pd.read_csv("data/df_files_UOHI.csv_metadata_extracted.csv")
display(df_uohi_extracted.describe())
df_uohi_extracted = pd.read_csv("data/df_files_MHI.csv_metadata_extracted.csv")
display(df_uohi_extracted.describe())
# df_uohi_extracted = pd.read_csv('data/df_files_UOHI.csv_metadata_extracted_mu.csv', sep='µ')
# display(df_uohi_extracted.describe())